# XBeach generate inputs

#### Initialize libraries

In [1]:
import os
import netCDF4 as nc4
import numpy as np
import pandas as pd
import pathlib as pl
from datetime import datetime,timedelta

### Initialize Paths to files and to output location

In [2]:
root = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/Jose/cfs') # 
path = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/xbeach/erratum/eslr_study/mgb')

#### Initialize Forcing time parameters

In [3]:
def find_node_ak(fmax,obs_lat,obs_lon):
    min_distance = None
    best_index = 0
    x = fmax.variables['x'][:]
    y = fmax.variables['y'][:]
    for i in range(len(x)):
        current_distance = (float(y[i]) - obs_lat)**2 + (float(x[i]) - obs_lon)**2
        if min_distance is None or current_distance < min_distance:
            best_index = i
            min_distance = current_distance
    #print("best_index:{} ".format(best_index))
    return best_index

In [4]:
def write_tide(root_dir,time,front,back):
    root_dir = pl.Path(root_dir)
    file = root_dir / 'tide.txt'
    data = []
    for i in range(0,len(time)):
        if float(front[i])/float(front[i]) != 1:
            if float(front[i-1])/float(front[i-1]) != 1:
                front[i] = 0.5
            else:
                front[i] = front[i-1]
        if float(back[i])/float(back[i]) != 1:
            if float(back[i-1])/float(back[i-1]) != 1:
                back[i] = 0.5
            else:
                back[i] = back[i-1]
         #   front[i] = 0
         #  back[i] = 0
        data.append('    {:.4e}    {:.4e}    {:.4e}'.format(float(time[i]),float(front[i]),float(back[i])) + '\n')
        #data.append('    {:.4e}    {:.4e}    {:.4e}'.format(float(time[i]),float(front[i]),float(back[i])*1.5) + '\n')
    with open(file,'w') as fin:
        fin.writelines(data)
    return

In [5]:
start = '20170910 00:00:00'
freq = '30min'
nc_file = nc4.Dataset(root / 'swan_HS.63.nc')
node = find_node_ak(nc_file,37.150951, -75.934593) # find the point closest to the coordinates specified
time = pd.date_range(start=start,periods=int(len(nc_file.variables['swan_HS'][:,node])),freq=freq)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/tmiesse/work/FHRL/eeslr/modelling/Jose/cfs/swan_HS.63.nc'

In [ ]:
start1,end1 ='2017-09-18 00:00:00','2017-09-20 00:00:00' #find the pts between the specified start and end times
model1 = time2.loc[(start1<=time2['time'])&(time2['time']<end1)].index.values

#### Read and Generate water level forcing for xbeach

In [53]:
h0 = nc4.Dataset(root / 'implicit0045' / 'swan_HS.63.nc','r').variables['swan_HS'][model1[0]:model1[-1],node]
tp = nc4.Dataset(root / 'explicit_cd1' /'swan_TPS.63.nc','r').variables['swan_TPS'][model1[0]:model1[-1],node]
main = nc4.Dataset(root / 'explicit_cd1' / 'swan_DIR.63.nc','r').variables['swan_DIR'][model1[0]:model1[-1],node]
waterlevel = nc4.Dataset(root / 'implicit0045' /'fort.63.nc').variables['zeta'][model1[0]:model1[-1],node]
interval = 1800
dt = np.arange(interval,len(h0)+interval,interval)

In [2]:
patha

NameError: name 'patha' is not defined

In [ ]:
#root_dir = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/xbeach/assateague/forcings')
#mainang=270.0
# mgb mainang=90 and es mainang=90

                    
write_tide(path,dt,waterlevel,waterlevel)
gammajsp=3.3
s=10.0
fnyq=0.45
timestep=1.0
data = []
period = (len(dt)*1800)+1800
duration = 1800
y = 0
print(d)
data.append('FILELIST'+'\n')
for i in range(1,int(period/duration)):
    with open(str(d / f'jonswap_{i}.txt'),'w') as fin:
        if (type(h0[y]) == np.ma.core.MaskedConstant) or (h0[y]==np.nan):
            h0[y]=0.001
            tp[y]=10
            main[y]=270
        elif (type(tp[y]) == np.ma.core.MaskedConstant) or (tp[y]==np.nan):
            tp[y]=5
            main[y]=270
        elif (type(main[y]) == np.ma.core.MaskedConstant) or (main[y]==np.nan):
            main[y]=270                                        
        fin.write('Hm0           = {:.4e}'.format(h0[y])+ '\n' +
                  'fp            = {:.4e}'.format(float(1/(tp[y])))+ '\n' +
                  'mainang       = {:.4e}'.format(main[y]) + '\n' +
                  'gammajsp      = {:.4e}'.format(gammajsp) + '\n' +
                  's             = {:.4e}'.format(s) + '\n' +
                  'fnyq          = {:.4e}'.format(fnyq) + '\n')
        data.append(f'    {duration}    {timestep}    jonswap_{i}.txt'+'\n')
    y+=1
with open(str(patha / 'filelist.txt'),'w') as control:
    lines = control.writelines(data)